<a href="https://colab.research.google.com/github/vedu-13/Codtech-Cyber-Security-/blob/main/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# It's best practice to save each class/module in its own file.
# For example, save the PortScanner class as 'port_scanner.py',
# the BruteForcer class as 'brute_forcer.py', and this main script
# as 'toolkit.py'. Then, you would use `from port_scanner import PortScanner`
# to import it. For this example, we'll keep them in one file for simplicity.

import socket
import hashlib
import itertools
import string
import time

# ==============================================================================
# MODULE 1: PORT SCANNER
# File: port_scanner.py
# ==============================================================================
class PortScanner:
    """
    A simple TCP port scanner to find open ports on a target host.
    """
    def __init__(self, target_ip):
        """
        Initializes the PortScanner.

        Args:
            target_ip (str): The IP address of the target to scan.
        """
        self.target_ip = target_ip
        self.open_ports = []

    def scan_port(self, port):
        """
        Scans a single port on the target IP.

        Args:
            port (int): The port number to check.

        Returns:
            bool: True if the port is open, False otherwise.
        """
        try:
            # Create a new socket using IPv4 and TCP
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            # Set a timeout to avoid waiting indefinitely for a response
            sock.settimeout(1)
            # Try to connect to the target IP and port
            result = sock.connect_ex((self.target_ip, port))
            # If the connection is successful (result is 0), the port is open
            if result == 0:
                return True
            sock.close()
        except socket.gaierror:
            # Hostname could not be resolved
            return False
        except socket.error:
            # Could not connect to server
            return False
        return False

    def scan(self, start_port, end_port):
        """
        Scans a range of ports on the target host.

        Args:
            start_port (int): The starting port of the scan range.
            end_port (int): The ending port of the scan range.
        """
        print(f"\n[+] Scanning target: {self.target_ip}")
        print(f"[*] Scanning ports from {start_port} to {end_port}...")

        self.open_ports = []
        for port in range(start_port, end_port + 1):
            if self.scan_port(port):
                self.open_ports.append(port)
                print(f"  [+] Port {port} is open")

        if not self.open_ports:
            print("\n[-] No open ports found in the specified range.")
        else:
            print(f"\n[+] Scan complete. Open ports: {self.open_ports}")

# ==============================================================================
# MODULE 2: BRUTE-FORCER (FOR HASH CRACKING)
# File: brute_forcer.py
# ==============================================================================
class BruteForcer:
    """
    A conceptual brute-forcer to crack a password hash.

    ETHICAL WARNING: This tool is for educational purposes only. Only use it
    on hashes for which you have explicit permission to test.
    """
    def __init__(self, target_hash, hash_algorithm='sha256'):
        """
        Initializes the BruteForcer.

        Args:
            target_hash (str): The hash string to crack.
            hash_algorithm (str): The hashing algorithm used (e.g., 'md5', 'sha256').
        """
        if not hasattr(hashlib, hash_algorithm):
            raise ValueError(f"Unsupported hash algorithm: {hash_algorithm}")

        self.target_hash = target_hash.lower()
        self.hash_algorithm = hash_algorithm

    def _check_password(self, password):
        """Hashes a password and compares it to the target hash."""
        hasher = hashlib.new(self.hash_algorithm)
        hasher.update(password.encode('utf-8'))
        return hasher.hexdigest() == self.target_hash

    def crack_with_wordlist(self, wordlist_path):
        """
        Attempts to crack the hash using a list of passwords from a file.

        Args:
            wordlist_path (str): The path to the password wordlist file.
        """
        print(f"\n[+] Starting wordlist attack on hash: {self.target_hash[:15]}...")
        try:
            with open(wordlist_path, 'r', encoding='utf-8', errors='ignore') as f:
                for line in f:
                    password = line.strip()
                    if self._check_password(password):
                        print(f"\n[!] SUCCESS! Password found: {password}")
                        return password
            print("\n[-] Wordlist exhausted. Password not found.")
        except FileNotFoundError:
            print(f"\n[!] ERROR: Wordlist file not found at '{wordlist_path}'")
        return None

    def crack_with_charset(self, max_length=8, charset=None):
        """
        Attempts to crack the hash using a generated charset.
        WARNING: This can be extremely slow for lengths > 6.

        Args:
            max_length (int): The maximum length of the password to try.
            charset (str): The character set to use for generating passwords.
        """
        if charset is None:
            charset = string.ascii_lowercase + string.digits

        print(f"\n[+] Starting character set attack (max length: {max_length})...")
        print(f"[*] Charset: '{charset}'")
        start_time = time.time()

        for length in range(1, max_length + 1):
            print(f"[*] Trying passwords of length {length}...")
            # Use itertools.product to generate all possible combinations
            for attempt in itertools.product(charset, repeat=length):
                password = "".join(attempt)
                if self._check_password(password):
                    end_time = time.time()
                    print(f"\n[!] SUCCESS! Password found: {password}")
                    print(f"[*] Time elapsed: {end_time - start_time:.2f} seconds")
                    return password

        end_time = time.time()
        print(f"\n[-] Character set attack complete. Password not found.")
        print(f"[*] Time elapsed: {end_time - start_time:.2f} seconds")
        return None


# ==============================================================================
# MAIN TOOLKIT INTERFACE
# File: toolkit.py
# ==============================================================================
def display_menu():
    """Prints the main menu."""
    print("\n" + "="*40)
    print("  Python Penetration Testing Toolkit")
    print("="*40)
    print("1. Port Scanner")
    print("2. Hash Cracker (Brute-Forcer)")
    print("3. Exit")
    print("-"*40)

def run_port_scanner():
    """Function to handle the port scanner logic."""
    target_ip = input("Enter the target IP address to scan: ")
    try:
        start_port_str = input("Enter the starting port (e.g., 1): ")
        end_port_str = input("Enter the ending port (e.g., 1024): ")
        start_port = int(start_port_str)
        end_port = int(end_port_str)

        if start_port > end_port or start_port < 0:
            print("[!] Invalid port range.")
            return

        scanner = PortScanner(target_ip)
        scanner.scan(start_port, end_port)
    except ValueError:
        print("[!] Invalid input. Please enter valid numbers for ports.")
    except Exception as e:
        print(f"[!] An unexpected error occurred: {e}")

def run_hash_cracker():
    """Function to handle the hash cracker logic."""
    target_hash = input("Enter the password hash to crack: ")
    hash_algo = input("Enter the hash algorithm (e.g., md5, sha1, sha256) [default: sha256]: ") or 'sha256'

    try:
        cracker = BruteForcer(target_hash, hash_algo)

        print("\nChoose cracking method:")
        print("  1. Wordlist Attack")
        print("  2. Character Set Attack (can be very slow)")
        choice = input("Enter your choice (1 or 2): ")

        if choice == '1':
            wordlist = input("Enter the path to your wordlist file: ")
            cracker.crack_with_wordlist(wordlist)
        elif choice == '2':
            max_len_str = input("Enter max password length to try (e.g., 5): ")
            max_len = int(max_len_str)
            cracker.crack_with_charset(max_len)
        else:
            print("[!] Invalid choice.")

    except ValueError as e:
        print(f"[!] Error: {e}")
    except Exception as e:
        print(f"[!] An unexpected error occurred: {e}")


if __name__ == "__main__":
    while True:
        display_menu()
        choice = input("Enter your choice (1-3): ")

        if choice == '1':
            run_port_scanner()
        elif choice == '2':
            run_hash_cracker()
        elif choice == '3':
            print("\n[+] Exiting toolkit. Stay safe!")
            break
        else:
            print("\n[!] Invalid choice. Please try again.")

        input("\nPress Enter to return to the menu...")


  Python Penetration Testing Toolkit
1. Port Scanner
2. Hash Cracker (Brute-Forcer)
3. Exit
----------------------------------------
Enter your choice (1-3): 2
Enter the password hash to crack: 3
Enter the hash algorithm (e.g., md5, sha1, sha256) [default: sha256]: 256
[!] Error: Unsupported hash algorithm: 256

Press Enter to return to the menu...

  Python Penetration Testing Toolkit
1. Port Scanner
2. Hash Cracker (Brute-Forcer)
3. Exit
----------------------------------------
Enter your choice (1-3): 3

[+] Exiting toolkit. Stay safe!
